In [1]:
##Ekstraksi Fitur Metode GLCM

In [2]:
import pandas as pd
import numpy as np
import cv2 
import os
import re
import matplotlib as mpl
import matplotlib.pyplot as plt

In [3]:
from skimage.feature import greycomatrix, greycoprops

input_matrix = np.array([[0, 0, 1],
                          [1, 2, 3],
                          [2, 3, 2]])

glcm = greycomatrix (input_matrix, 
                    distances=[1], 
                    angles=[0], 
                    levels=4,
                    symmetric=True, 
                    normed=True)

In [4]:
print(glcm[:,:,0,0])

[[0.16666667 0.08333333 0.         0.        ]
 [0.08333333 0.         0.08333333 0.        ]
 [0.         0.08333333 0.         0.25      ]
 [0.         0.         0.25       0.        ]]


In [5]:
## Sediakan folder training

In [6]:
# -------------------- Utility function ------------------------
def normalize_label(str_):
    str_ = str_.replace(" ", "")
    str_ = str_.translate(str_.maketrans("","", "()"))
    str_ = str_.split("_")
    return ''.join(str_[:2])

def normalize_desc(folder, sub_folder):
    text = folder + " - " + sub_folder 
    text = re.sub(r'\d+', '', text)
    text = text.replace(".", "")
    text = text.strip()
    return text

def print_progress(val, val_len, folder, sub_folder, filename, bar_size=10):
    progr = "#"*round((val)*bar_size/val_len) + " "*round((val_len - (val))*bar_size/val_len)
    if val == 0:
        print("", end = "\n")
    else:
        print("[%s] folder : %s/%s/ ----> file : %s" % (progr, folder, sub_folder, filename), end="\r")
        

# -------------------- Load Dataset ------------------------
 
dataset_dir = "C:\\Users\\Posma\\Downloads\\SKrpsi\\skins_training\\"

imgs = [] #list image matrix 
labels = []
descs = []
for folder in os.listdir(dataset_dir):
    for sub_folder in os.listdir(os.path.join(dataset_dir, folder)):
        sub_folder_files = os.listdir(os.path.join(dataset_dir, folder, sub_folder))
        len_sub_folder = len(sub_folder_files) - 1
        for i, filename in enumerate(sub_folder_files):
            img = cv2.imread(os.path.join(dataset_dir, folder, sub_folder, filename))
            
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            h, w = gray.shape
            ymin, ymax, xmin, xmax = h//3, h*2//3, w//3, w*2//3
            crop = gray[ymin:ymax, xmin:xmax]
            
            resize = cv2.resize(crop, (0,0), fx=0.5, fy=0.5)
            
            imgs.append(resize)
            labels.append(normalize_label(os.path.splitext(filename)[0]))
            descs.append(normalize_desc(folder, sub_folder))
            
            print_progress(i, len_sub_folder, folder, sub_folder, filename)



[##########] folder : training/chickenpox/ ----> file : ISIC_0073247.jpgjpg
[##########] folder : training/dermatofibroma/ ----> file : ISIC_0070546.jpg
[##########] folder : training/eczema/ ----> file : ISIC_0069849.jpg
[##########] folder : training/keratosis/ ----> file : ISIC_9811367.jpgg
[##########] folder : training/psoriasis/ ----> file : ISIC_0016069.jpg


In [7]:
cv2.imshow("train img", imgs[0])

cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
from skimage.feature import greycomatrix, greycoprops

# ----------------- calculate greycomatrix() & greycoprops() for angle 0, 45, 90, 135 ----------------------------------
def calc_glcm_all_agls(img, label, props, dists=[5], agls=[0, np.pi/4, np.pi/2, 3*np.pi/4], lvl=256, sym=True, norm=True):
    
    glcm = greycomatrix(img, 
                        distances=dists, 
                        angles=agls, 
                        levels=lvl,
                        symmetric=sym, 
                        normed=norm)
    feature = []
    glcm_props = [propery for name in props for propery in greycoprops(glcm, name)[0]]
    for item in glcm_props:
            feature.append(item)
    feature.append(label) 
    
    return feature


# ----------------- call calc_glcm_all_agls() for all properties ----------------------------------
properties = ['dissimilarity', 'correlation', 'homogeneity', 'contrast', 'ASM', 'energy']

glcm_all_agls = []
for img, label in zip(imgs, labels): 
    glcm_all_agls.append(
            calc_glcm_all_agls(img, 
                                label, 
                                props=properties)
                            )
 
columns = []
angles = ['0', '45', '90','135']
for name in properties :
    for ang in angles:
        columns.append(name + "_" + ang)
        
columns.append("label")


In [9]:
import pandas as pd 
# Create the pandas DataFrame for GLCM features data
glcm_df = pd.DataFrame(glcm_all_agls, 
                      columns = columns)

#save to csv
glcm_df.to_csv("glcm_training_dataset.csv")

glcm_df.head(6)

,dissimilarity_0,dissimilarity_45,dissimilarity_90,dissimilarity_135,correlation_0,correlation_45,correlation_90,correlation_135,homogeneity_0,homogeneity_45,...,contrast_135,ASM_0,ASM_45,ASM_90,ASM_135,energy_0,energy_45,energy_90,energy_135,label
0,5.434225,5.856002,4.859893,5.508129,0.802098,0.781001,0.832751,0.790281,0.201666,0.180426,...,71.377268,0.001236,0.001139,0.001346,0.001241,0.035154,0.033756,0.036681,0.035229,ISIC0071991
1,10.579964,11.126325,9.944064,10.797104,0.603548,0.570257,0.646676,0.587455,0.129391,0.121874,...,250.674082,0.000549,0.000521,0.000570,0.000530,0.023422,0.022834,0.023866,0.023019,ISIC0071999
2,7.705169,8.149260,7.454652,7.934352,0.902226,0.892066,0.903614,0.890765,0.133669,0.128542,...,123.708485,0.000510,0.000486,0.000539,0.000507,0.022580,0.022044,0.023213,0.022526,ISIC0072023
3,3.488164,3.839490,3.656114,3.762447,0.812095,0.756053,0.779431,0.784832,0.274938,0.268350,...,28.394832,0.003219,0.003123,0.003282,0.003068,0.056733,0.055884,0.057286,0.055386,ISIC0072028
4,5.890624,6.064487,5.625062,6.283786,0.803134,0.787744,0.798775,0.770924,0.183727,0.175526,...,88.763972,0.000935,0.000910,0.000994,0.000885,0.030584,0.030174,0.031535,0.029757,ISIC0072029
5,7.831907,8.481057,8.264528,8.548519,0.572918,0.509788,0.542219,0.507493,0.150246,0.135163,...,158.155792,0.000949,0.000886,0.000896,0.000872,0.030798,0.029758,0.029927,0.029537,ISIC0072034


In [10]:
## Sediakan folder testing

In [11]:
# -------------------- Utility function ------------------------
def normalize_label(str_):
    str_ = str_.replace(" ", "")
    str_ = str_.translate(str_.maketrans("","", "()"))
    str_ = str_.split("_")
    return ''.join(str_[:2])

def normalize_desc(folder, sub_folder):
    text = folder + " - " + sub_folder 
    text = re.sub(r'\d+', '', text)
    text = text.replace(".", "")
    text = text.strip()
    return text

def print_progress(val, val_len, folder, sub_folder, filename, bar_size=10):
    progr = "#"*round((val)*bar_size/val_len) + " "*round((val_len - (val))*bar_size/val_len)
    if val == 0:
        print("", end = "\n")
    else:
        print("[%s] folder : %s/%s/ ----> file : %s" % (progr, folder, sub_folder, filename), end="\r")
        

# -------------------- Load Dataset ------------------------
 
dataset_dir = "C:\\Users\\Posma\\Downloads\\SKrpsi\\skins_test\\"

imgs = [] #list image matrix 
labels = []
descs = []
for folder in os.listdir(dataset_dir):
    for sub_folder in os.listdir(os.path.join(dataset_dir, folder)):
        sub_folder_files = os.listdir(os.path.join(dataset_dir, folder, sub_folder))
        len_sub_folder = len(sub_folder_files) - 1
        for i, filename in enumerate(sub_folder_files):
            img = cv2.imread(os.path.join(dataset_dir, folder, sub_folder, filename))
            
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            h, w = gray.shape
            ymin, ymax, xmin, xmax = h//3, h*2//3, w//3, w*2//3
            crop = gray[ymin:ymax, xmin:xmax]
            
            resize = cv2.resize(crop, (0,0), fx=0.5, fy=0.5)
            
            imgs.append(resize)
            labels.append(normalize_label(os.path.splitext(filename)[0]))
            descs.append(normalize_desc(folder, sub_folder))
            
            print_progress(i, len_sub_folder, folder, sub_folder, filename)



[##########] folder : evaluation/chickenpox/ ----> file : ISIC_0072833.jpg
[##########] folder : evaluation/dermatofibroma/ ----> file : ISIC_0029052.jpg
[##########] folder : evaluation/eczema/ ----> file : ISIC_0055583.jpg
[##########] folder : evaluation/keratosis/ ----> file : ISIC_2371734.jpeg
[##########] folder : evaluation/psoriasis/ ----> file : ISIC_0014473.jpg


In [12]:
import pandas as pd 
# Create the pandas DataFrame for GLCM features data
glcm_df = pd.DataFrame(glcm_all_agls, 
                      columns = columns)

#save to csv
glcm_df.to_csv("glcm_test_dataset.csv")

glcm_df.head(6)

,dissimilarity_0,dissimilarity_45,dissimilarity_90,dissimilarity_135,correlation_0,correlation_45,correlation_90,correlation_135,homogeneity_0,homogeneity_45,...,contrast_135,ASM_0,ASM_45,ASM_90,ASM_135,energy_0,energy_45,energy_90,energy_135,label
0,5.434225,5.856002,4.859893,5.508129,0.802098,0.781001,0.832751,0.790281,0.201666,0.180426,...,71.377268,0.001236,0.001139,0.001346,0.001241,0.035154,0.033756,0.036681,0.035229,ISIC0071991
1,10.579964,11.126325,9.944064,10.797104,0.603548,0.570257,0.646676,0.587455,0.129391,0.121874,...,250.674082,0.000549,0.000521,0.000570,0.000530,0.023422,0.022834,0.023866,0.023019,ISIC0071999
2,7.705169,8.149260,7.454652,7.934352,0.902226,0.892066,0.903614,0.890765,0.133669,0.128542,...,123.708485,0.000510,0.000486,0.000539,0.000507,0.022580,0.022044,0.023213,0.022526,ISIC0072023
3,3.488164,3.839490,3.656114,3.762447,0.812095,0.756053,0.779431,0.784832,0.274938,0.268350,...,28.394832,0.003219,0.003123,0.003282,0.003068,0.056733,0.055884,0.057286,0.055386,ISIC0072028
4,5.890624,6.064487,5.625062,6.283786,0.803134,0.787744,0.798775,0.770924,0.183727,0.175526,...,88.763972,0.000935,0.000910,0.000994,0.000885,0.030584,0.030174,0.031535,0.029757,ISIC0072029
5,7.831907,8.481057,8.264528,8.548519,0.572918,0.509788,0.542219,0.507493,0.150246,0.135163,...,158.155792,0.000949,0.000886,0.000896,0.000872,0.030798,0.029758,0.029927,0.029537,ISIC0072034
